# Challenge Scratchbook

* This notebook explores methods for the Kernel Methods for Machine Learning Kaggle [challenge](https://www.kaggle.com/c/kernel-methods-for-machine-learning-2018-2019/data).

* Note that this is a binary classification challenge.

Our first goal is to implement two baseline methods:
1. Random classification
2. All instances are 0s (Doing so we get an idea of the proportion of 0's in the public test set)
3. Implement the Simple Pattern Recognition Algorithm (SPR) from Learning with Kernels 

Before that, we have to implement some data loaders

## Imports

In [13]:
import csv
import os
import numpy as np

In [1]:
# TODO: Implement a data loader for raw CSV files

# TODO: Implement a data loader for array CSV files

# TODO: Implement a data saver


## Paths and Globals

In [9]:
CWD = os.getcwd()
DATA_DIR = os.path.join(CWD, "data")
RESULT_DIR = os.path.join(CWD, "results")

FILES = {0: {"train_mat": "Xtr0_mat100.csv",
             "train": "Xtr0.csv",
             "test_mat": "Xte0_mat100.csv",
             "test": "Xte0.csv",
             "label": "Ytr0.csv"},
         1: {"train_mat": "Xtr1_mat100.csv",
             "train": "Xtr1.csv",
             "test_mat": "Xte1_mat100.csv",
             "test": "Xte1.csv",
             "label": "Ytr1.csv"},
         2: {"train_mat": "Xtr2_mat100.csv",
             "train": "Xtr2.csv",
             "test_mat": "Xte2_mat100.csv",
             "test": "Xte2.csv",
             "label": "Ytr2.csv"}}

## 0 entries

In [13]:
with open(os.path.join(RESULT_DIR, "results.csv"), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    writer.writerow(["Id", "Bound"])
    for i in range(3000):
        writer.writerow([i, 0])

**Comment:**

* We get 0.51266 which means that the dataset is pretty balanced.

## SPR

In [37]:
class SPR:
    """
    This class implements the Simple Pattern Recognition algorithm found in the Learning with Kernel books
    """
    def __init__(self):
        self.m0 = 0
        self.m1 = 0
        self.b = 0

        
    def fit(self,X,y):
        """
        Fitting phase
        
        Parameters
        ------------
        - X : numpy.ndarray
            Data matrix
            
        - y : numpy.array
            Labels
        """
        
        self.X0 = X[y == 0]
        self.X1 = X[y == 1]
        
        self.m0 = len(self.X0)
        self.m1 = len(self.X1)
        
        self.b = 1/2 * (1/(self.m0**2)*np.sum(self.X0.dot(self.X0.T)) - 1/(self.m1**2)*np.sum(self.X1.dot(self.X1.T)))
        
        #self.b = 1/2 * (1/(self.m0**2)*np.sum([kernel(self.X_train[i],self.X_train[j]) for i in self.list0 for j in self.list0]) - 1/(self.m1)**2*np.sum([kernel(self.X_train[i],self.X_train[j]) for i in self.list1 for j in self.list1]))
    
    def predict(self,X):
        
        y_pred = np.zeros(len(X))
        
        for i in range(len(X)):
            val = 1 / self.m0 * np.sum(self.X0.dot(X[i])) - 1 / self.m1 * np.sum(self.X1.dot(X[i])) + self.b
            #val = 1/self.m1*np.sum([kernel(self.X_train[k],X[i]) for k in self.list1]) - 1/self.m0*np.sum([kernel(self.X_train[k],X[i]) for k in self.list0]) + self.b
            y_pred[i] = np.sign(val)/2 + 1/2
        return y_pred
    
        #val = 1 / self.m0 * np.sum(self.X0.dot(X.T)) + 1 / self.m1 * np.sum(self.X1.dot(X.T)) + self.b
        #y_pred = np.sign(val) / 2 + 1/2
        #return y_pred.astype("int")
    
    
    
    def score(self, y, y_pred):
        return np.sum(y[y == y_pred]) / len(y)
    

## Data loading refactoring

In [38]:
def load_data(file_id, mat=True):
    
    X_train = list()
    Y_train = list()
    X_test = list()
    
    dic = FILES[file_id]
    
    if mat:
        files = [dic["train_mat"], dic["label"], dic["test_mat"]]
    else:
        files = [dic["train"], dic["label"], dic["test"]]

    for file, l in zip(files, [X_train, Y_train, X_test]):
        with open(os.path.join(DATA_DIR, file), "r", newline="") as csvfile:
            if "Y" in file:
                reader = csv.reader(csvfile, delimiter=",")
                next(reader, None) # Skip the header
                for row in reader:
                    l.append(row[1])
            else:
                reader = csv.reader(csvfile, delimiter=" ")
                for row in reader:
                    l.append(row)
                
    X_train = np.array(X_train).astype("float")
    Y_train = np.array(Y_train).astype("int")
    X_test = np.array(X_test).astype("float")
    
    return X_train, Y_train, X_test

## Train and test on the different sets

In [41]:
results = np.zeros(3000)

for i in range(len(FILES)):
    X_train, Y_train, X_test = load_data(i)
    clf = SPR()
    clf.fit(X_train, Y_train)
    results[i*1000:i*1000 + 1000] = clf.predict(X_test)

## Save results

In [47]:
def save_results(filename, results):
    """
    Save results in a csv file
    
    Parameters
    -----------
    - filename : string
        Name of the file to be saved under the ``results`` folder
        
    - results : numpy.array
        Resulting array (0 and 1's)
    """
    
    assert filename.endswith(".csv"), "this is not a csv extension!"
    # Convert results to int
    results = results.astype("int")
    
    with open(os.path.join(RESULT_DIR, filename), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        # Write header
        writer.writerow(["Id", "Bound"]) 
        assert len(results) == 3000, "There is not 3000 predictions"
        # Write results
        for i in range(len(results)):
            writer.writerow([i, results[i]])

In [46]:
# Test the save results function
save_results("test_results.csv", results)